<a href="https://colab.research.google.com/github/kentaojapi/dl-began-1st-try/blob/master/BEGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
BUCKET_NAME = 'began-first-ojapi'
INPUT_SIZE = (64, 64, 3)

In [0]:
import numpy as np
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, load_img, array_to_img
from tensorflow.python.keras.layers import Dense, Conv2D, UpSampling2D, Input, Reshape, Flatten

from tensorflow.python.keras.losses import mean_absolute_error
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras import backend as K

In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

In [0]:
!mkdir ../content/img/
!mkdir ../content/img_resized/

mkdir: cannot create directory ‘../content/img/’: File exists
mkdir: cannot create directory ‘../content/img_resized/’: File exists


In [0]:
from google.cloud import storage


def list_blobs(bucket_name):
    """
    get lists all the blobs in the bucket.
    refer : https://github.com/GoogleCloudPlatform/python-docs-samples/blob/master/storage/cloud-client/snippets.py 
    """
    storage_client = storage.Client('My First Project')
    bucket = storage_client.get_bucket(bucket_name)
    blobs = bucket.list_blobs()
    imgs_name_list = []
    for blob in blobs:
        imgs_name_list.append(blob.name)
    return imgs_name_list
  
  
def download_blob(bucket_name, source_blob_name, destination_file_name):
    """
    Downloads a blob from the bucket.
    refer : https://github.com/GoogleCloudPlatform/python-docs-samples/blob/master/storage/cloud-client/snippets.py
    """
    storage_client = storage.Client('My First Project')
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    blob.download_to_filename(destination_file_name)

    print('Blob {} downloaded to {}.'.format(
        source_blob_name,
        destination_file_name))


imgs_name_list = list_blobs(BUCKET_NAME)
download_counts = 0 
error_counts = 0
for img_name in imgs_name_list:
  download_blob(BUCKET_NAME, '{}'.format(img_name),  '../content/img/{}'.format(img_name))
  try:
    img_resized = load_img('../content/img/{}'.format(img_name), target_size=INPUT_SIZE)
    img_resized.save('../content/img_resized/{}'.format(img_name))  
    download_counts += 1
  except OSError:
    error_counts += 1
    pass
  
  
from IPython.display import clear_output
clear_output()


print('download counts:{}'.format(download_counts))
print('error counts: {}'.format(error_counts))

download counts:3120
error counts: 19


In [0]:
DATA_DIR = '../content/'
BATCH_SIZE = 16
IMG_SHAPE = (64, 64, 3)

data_gen = ImageDataGenerator(rescale=1/255.)
train_data_generator = data_gen.flow_from_directory(
    directory=DATA_DIR,
    classes=['img_resized'],
    class_mode=None,
    batch_size=BATCH_SIZE,
    target_size=IMG_SHAPE[:2]
)

Found 3120 images belonging to 1 classes.


In [0]:
!ls ../content/img_resized/

n09618957_10000.jpg  n09918248_14825.jpg  n10289039_13029.jpg
n09618957_10005.jpg  n09918248_1483.jpg   n10289039_13045.jpg
n09618957_10023.jpg  n09918248_14896.jpg  n10289039_13067.jpg
n09618957_10042.jpg  n09918248_14926.jpg  n10289039_1306.jpg
n09618957_10062.jpg  n09918248_14935.jpg  n10289039_1307.jpg
n09618957_10065.jpg  n09918248_1496.jpg   n10289039_13081.jpg
n09618957_10069.jpg  n09918248_15009.jpg  n10289039_13091.jpg
n09618957_10075.jpg  n09918248_15011.jpg  n10289039_13094.jpg
n09618957_1007.jpg   n09918248_15054.jpg  n10289039_13096.jpg
n09618957_10097.jpg  n09918248_15104.jpg  n10289039_13122.jpg
n09618957_10113.jpg  n09918248_15123.jpg  n10289039_13127.jpg
n09618957_10132.jpg  n09918248_15184.jpg  n10289039_13129.jpg
n09618957_10134.jpg  n09918248_15191.jpg  n10289039_13153.jpg
n09618957_10182.jpg  n09918248_15250.jpg  n10289039_13168.jpg
n09618957_10212.jpg  n09918248_15255.jpg  n10289039_13171.jpg
n09618957_10250.jpg  n09918248_15270.jpg  n10289039_13180.jpg
n09618957_

In [0]:
def build_encoder(input_shape, z_size, n_filters, n_layers):
  """
  build encoder.
  
  Arguments:
    input_shape(int): shape of imgs
    z_size(int): the number of dimensions in a feature space
    n_filters(int): the number of fileters
    
  Returns:
    model(Model): Encode Model
  """
  model = Sequential()
  model.add(
      Conv2D(
          n_filters, 
          3,
          activation='elu',
          input_shape=input_shape,
          padding='same'
      )
  )
  model.add(Conv2D(n_filters, 3, padding='same'))
  for i in range(2, n_layers+1):
    model.add(
        Conv2D(
            n_filters,
            3,
            activation='elu',
            input_shape=input_shape,
            padding='same'
        )
    )
    model.add(
        Conv2D(
            i*n_filters,
            3,
            activation='elu',
            strides=2,
            padding='same'
        )
    )
    model.add(Conv2D(n_layers*n_filters, 3, padding='same'))
    model.add(Flatten())
    model.add(Dense(z_size))

    
    return model





In [0]:
def build_decoder(output_shape, z_size, n_filters, n_layers):
  """
  build decoder
  
  Arguments:
    output_shape(np.array): shape of imgs
    z_size(int) : the number of dimensions in a feature space
    n_filters(int): the number of filters
    n_layers(int): the number of layers
    
  Returns:
    model(Model): Decoder Model
  """
  # how times it will be expanded with UpSampling2D.
  scale = 2**(n_layers -1)
  # calculate first input shape of the first concovolutional layer from scale
  fc_shape = (
      output_shape[0]//scale,
      output_shape[1]//scale,
      n_filters
  )
  # calcurate back shape of fully connected layer
  fc_size = fc_shape[0]*fc_shape[1]*fc_shape[2]
  
  # connected layer
  model = Sequential()
  model.add(Dense(fc_size, input_shape=(z_size, )))
  model.add(Reshape(fc_shape))
  
  # repeat covolutional layers
  for i in range(n_layers - 1):
    model.add(
        Conv2D(
            n_filters,
            3,
            activation='elu',
            padding='same'
        )
    )
    model.add(
        Conv2D(
            n_filters,
            3,
            activation='elu',
            padding='same'
        )
    )
    model.add(UpSampling2D())
    
  # last layer is not necessary for UpSampling2D
  model.add(
      Conv2D(
          n_filters,
          3,
          activation='elu',
          padding='same'
      )
  )
  model.add(
      Conv2D(
          n_filters,
          3,
          activation='elu',
          padding='same'
      )
  )
  # transform to 3 channels at output_layer
  model.add(Conv2D(3, 3, padding='same'))
    
  return model

In [0]:
def build_generator(img_shape, z_size, n_filters, n_layers):
  decoder = build_decoder(
      img_shape, z_size, n_filters, n_layers
  )
  return decoder

In [0]:
def build_discriminator(img_shape, z_size, n_filters, n_layres):
  encoder = build_encoder(
      img_shape, z_size, n_filters, n_layres
  )
  decoder = build_decoder(
      img_shape, z_size, n_filters, n_layers
  )
  return Sequential((encoder, decoder))

In [0]:
def build_discriminator_trainer(discriminator):
  img_shape = discriminator.input_shape[1:]
  real_inputs = Input(img_shape)
  fake_inputs = Input(img_shape)
  real_outputs = discriminator(real_inputs)
  fake_outputs = discriminator(fake_inputs)
  
  return Model(
      inputs=[real_inputs, fake_inputs],
      outputs=[real_outputs, fake_outputs]
  )

In [0]:
# build a network.
n_filters = 64
n_layers = 4
z_size = 32

generator = build_generator(
    IMG_SHAPE, z_size, n_filters, n_layers
)
discriminator = build_discriminator(
    IMG_SHAPE, z_size, n_filters, n_layers
)
discriminator_trainer = build_discriminator_trainer(
    discriminator
)

generator.summary()
# discriminator.layers[1] is Decoder
discriminator.layers[1].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4096)              135168    
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 8, 64)          36928     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        36928     
__________

In [0]:
def build_generator_loss(discriminator):
  # define a loss, using discriminator.
  def loss(y_ture, y_pred):
    # y_true is dummy.
    reconst = discriminator(y_pred)
    return mean_absolute_error(
        reconst,
        y_pred
    )
  return loss

In [0]:
# An initial learnig rate.(Generator)
g_lr = 0.0001

generator_loss = build_generator_loss(discriminator)
generator.compile(
    loss=generator_loss,
    optimizer=Adam(g_lr)
)

In [0]:
# An inital learning rate. (Discriminator)
d_lr = 0.0001

# k_var is int
k_var = 0.0
# k is a Variable of Keras(Tensorflow)
k = K.variable(k_var)
discriminator_trainer.compile(
    loss=[
        mean_absolute_error,
        mean_absolute_error
    ],
    loss_weights=[1., -k],
    optimizer=Adam(d_lr)
)

In [0]:
def measure(real_loss, fake_loss, gamma):
  return real_loss + np.abs(gamma*real_loss - fake_loss)

In [0]:
## Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# a paramater to update k
GAMMA = 0.5
LR_K = 0.001

# total steps(decide about from 100,000 to 1,000,000)
TOTAL_STEPS = 100000

# a directory to save models and generated imgs.
MODEL_SAVE_DIR = "../content/drive/My Drive/machine_learning/began/model/"
IMG_SAVE_DIR = "../content/drive/My Drive/machine_learning/began/imgs/"
# generate 5*5 imgs to check.
IMG_SAMPLE_SHAPE = (5, 5)
N_IMG_SMAPLES = np.prod(IMG_SMAPLE_SHAPE)

# if there is not save dirs, make them.
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
os.makedirs(IMG_SAVE_DIR, exist_ok=True)

In [0]:
def save_imgs(path, imgs, rows, cols):
    base_width = imgs.shape[1]
    base_height = imgs.shape[2]
    channels = imgs.shape[3]
    output_shape = (
        base_height*rows,
        base_width*cols,
        channels
    )
    buffer = np.zeros(output_shape)
    for row in range(rows):
        for col in range(cols):
            img = imgs[row*cols + col]
            buffer[
                row*base_height:(row + 1)*base_height,
                col*base_width:(col + 1)*base_width
            ] = img
    array_to_img(buffer).save(path)

In [0]:
# random seeds to generate sample imgs.
sample_seeds = np.random.uniform(
    -1, 1, (N_IMG_SMAPLES, z_size)
)

history = []
logs = []

for step, batch in enumerate(train_data_generator):
  # if a number of samples is less than BATCH_SIZE, skip.
  # if Total number of imgs is not multiple of BATCH_SIZE.
  if len(batch) < BATCH_SIZE:
    continue
    
  # stop a learning.
  if step > TOTAL_STEPS:
    break
    
  # generate random tensors.
  z_g = np.random.uniform(
      -1, 1, (BATCH_SIZE, z_size)
  )
  
  z_d = np.random.uniform(
      -1, 1, (BATCH_SIZE, z_size)
  )
  
  # generated imgs(for learning of discriminator).
  g_pred = generator.predict(z_d)
  
  # learn in a one step for generator.
  generator.train_on_batch(z_g, batch)
      
  # learn in a one step for discriminator.
  _, real_loss, fake_loss = discriminator_trainer.train_on_batch(
      [batch, g_pred], 
      [batch, g_pred]
  )
  
  # update k
  k_var += LR_K*(GAMMA*real_loss - fake_loss)
  K.set_value(k, k_var)
  
  # save loss to calcurate g_measure.
  history.append({
      'real_loss': real_loss,
      'fake_loss': fake_loss
  })
  
  # show a log once in 1000times.
  if step%1000 == 0:
    # average measures of past 1000 times.
    measurement = np.mean([
        measure(
            loss['real_loss'],
            loss['fake_loss'],
            GAMMA
        )
        for loss in history[-1000:]
    ])
    
    logs.append({
        'k' : K.get_value(k),
        'real_loss' : real_loss,
        'fake_loss' : fake_loss
    })
    print(logs[-1])
    
    img_path = '{}/generated_{}.png'.format(IMG_SAVE_DIR, step)
    save_imgs(
        img_path,
        generator.predict(sample_seeds),
        rows=IMG_SAMPLE_SHAPE[0],
        cols=IMG_SMAPLE_SHAPE[1]
    )
    # save a latest model
    generator.save('{}/generator_{}.hd5'.format(MODEL_SAVE_DIR, step))
    discriminator.save('{}/discriminator_{}.hd5'.format(MODEL_SAVE_DIR, step))  

{'k': 0.0136474045, 'real_loss': 1.1731069, 'fake_loss': 0.6236351}


KeyboardInterrupt: ignored

In [0]:
train_data_generator.show()

AttributeError: ignored